In [10]:
cardsToValues = {'A': 1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 'J':10, 'Q':10, 'K':10}
valuesToCards = {1:'A' , 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, 10:10, 11:'J', 12:'Q', 13:'K'}
getCard = lambda value : valuesToCards[value]
getValue = lambda card : cardsToValues[card]
class Cards (object) :
    
    def __init__ (self,s, v) :
        self.suit = s
        self.value = v
    def __str__ (self) :
        return ("{v} of {s}").format(v = self.value, s = self.suit)
        

import random
class Deck (object) :
    suits = ['Heart', 'Club', 'Diamond', 'Spade']
    size = int()

    def __init__ (self, s) :
        self.size = s 
        valSuit = s / 4
        self.deck = list()
        #Create deck of equal parts
        for suit in self.suits :
            for i in xrange(1, valSuit + 1) :
                self.deck.append(Cards(suit,  getCard(i)))
        random.shuffle(self.deck)

    def deal (self, num) :
        self.size -= num
        hand = [self.deck.pop() for i in xrange(0 , num)]
        return hand
    
    def hit (self) :
        self.size -= 1
        return self.deck.pop()
        

class Player (object) :
    hand = list()

    def __init__ (self, bank=100, name='Player') :
        self.bankroll = bank
        self.pName = name
        self.wins = 0
        self.losses = 0
    
    def clear (self) :
        hand = []

def sumHand (hand) :
    oneFlag = False
    oneSet = set()
    result = 0
    fin = True
    while fin :
        try :
            for card in hand :
                if (card.value == 'A' and not oneFlag and not card.suit in oneSet) :
                    oneFlag = True
                    oneSet.add(card.suit)
                    result += 11
                    continue
                result += getValue(card.value)
            if (result > 21 and oneFlag) :
                result = 0
                oneFlag = False
                raise OverLoad
        except OverLoad :
                continue
        else :
            fin = False
    
    return result

def dealerBet (dealer, player, playerBet) :
    dealerVal = sumHand(dealer.hand)
    playerVal = sumHand(player.hand)
    myBet = 0
    if dealerVal >= playerVal:
        if dealerVal >= 15:
            myBet = playerBet + playerBet * .5
        elif dealerVal < 10 :
            myBet = playerBet
        else :
            myBet = playerBet + playerBet * .25
    else :
        #Bluff
        if dealer.wins > player.wins and  playerVal - dealerVal >= 10 and dealerVal < 10:
            myBet = playerBet * 3
        elif playerVal - dealerVal < 5 :
            x = -1 if dealer.wins % 2 == 0 else 1
            myBet = playerBet + playerBet * .2 * x
        elif playerVal >= 16 and dealerVal < 10:
            myBet = 0
        else :
            x = -1 if player.wins > dealer.wins else 1
            myBet = playerBet + playerBet * .1 * x
    return myBet

def dealerPlay (dealer, player, currentPot, winAmount, deck, stat) :
    dealerAmount = sumHand(dealer.hand)
    playerAmount = sumHand(player.hand)
    dStat = 'c'
    finalDecision = 'D'
    if playerAmount  > dealerAmount:
        while dStat == 'c' :
            if stat == 'w' or player.bankroll + currentPot or dealerAmount < 12:
                dealer.hand.append(deck.hit())
            else :
                x = True if ((16 - numTen) / (len(deck) * 1.0)) <= .5 else False 
                if dealer.wins > 5 and not x:
                    dealer.hand.append(deck.hit())
                elif dealer.wins < player.wins and x :
                    dealer.hand.append(deck.hit())
                else :
                    dStat = 'f'
            dealerAmount = sumHand(dealer.hand)
            if dealerAmount > 21 :
                dStat = 'b'
            elif dealerAmount >= playerAmount :
                dStat = 'w'
        if dStat == 'b' or dStat == 'f':
            finalDecision = 'L'
    return finalDecision

def printBoard (player, dealer, currentPot, amountWin) :
    printingHand = [str(card) for card in player.hand]
    print("\t\t\t|\n\t\t\t|\n\t\t\t|\n\t\t\t|\n\t\t\t|")
    print("-------------------------------------------------------------------------")
    print("Current Pot : {x}  |        | Dealer |          Dealer Wins   = {w}").format(x = currentPot, w = dealer.wins)
    print("-------------------~                             Dealer Losses = {l}").format(l = dealer.losses)
    print("                                                 Dealer Bank   = {a}").format(a = dealer.bankroll)
    print("                          Amount to Win                             ")
    print("                                {a}                                 ").format(a = amountWin)
    print("{n} Wins   = {w}                                                    ").format(n = player.pName, w = player.wins)
    print("{n} Losses = {l}                                                    ").format(n = player.pName, l = player.losses)
    print("{n} Bank   = {a}                                                    ").format(n = player.pName, a = player.bankroll)
    print("Your Hand :                                                         ")
    print(printingHand)
    print("-------------------------------------------------------------------------")   

class InvalidInput (Exception) :
    pass

class InvalidBet (Exception) :
    pass

class PlayAgain (Exception) :
    pass
class OverLoad (Exception):
    pass

#Make PrintBoard
def blackjack () :
    name = raw_input("Please enter your name ")
    fin = True
    pInitialBank = 0
    while fin :
        try :
            pInitialBank = int(raw_input("How much money would you like to start with? "))
        except :
            print ("Please enter an Integer")
            continue
        else :
            fin = False
            print ("Thank you {0} your starting bank will be {1}").format(name, pInitialBank)

    fin = True
    while fin :
        try:
            winAmount = int(raw_input("What dollar amount would you like to play to "))
            if winAmount <= pInitialBank :
                print ('Please enter an amount that is greater than your initial amount')
                continue
        except ValueError:
            print ('Please enter an Integer')
        else:
            fin = False
            print ('Thank you the game will be played until you get %d or more or go broke') %(winAmount)


    dealer = Player(winAmount * 3, 'Dealer')
    player = Player(pInitialBank, name)
    while True :
        try :
            print ("Dealer wins ties")
            player.bankroll = pInitialBank
            dealer.bankroll = winAmount * 3
            deck = Deck(52)

            while player.bankroll < winAmount and  player.bankroll > 0 and dealer.bankroll > 0:
                if deck.size <= 2 :
                    print ("Reshuffling -----~")
                    deck = Deck(52)
                currentPot = 0
                player.hand = deck.deal(2)
                dealer.hand = deck.deal(2)
                if deck.size == 0 :
                    print ("Reshuffling -----~")
                    deck = Deck(52)

                printBoard(player, dealer, currentPot, winAmount)
                print ("Here is the current value of your hand {x}").format(x = sumHand(player.hand))
                print ("This is what you can see of the dealer's hand\n* {x}").format(x = dealer.hand[1])

                bet = 0
                stat = 'c'
                while True :
                    try :
                        print("How much would you like to bet?")
                        bet = raw_input("If you would like to fold input 0 or if you'd like to go All In input 'All In' : ").lower()
                        if bet == 'all in':
                            bet = player.bankroll
                        else :
                            bet = float(bet)
                        if bet < 0 or bet > player.bankroll :
                            raise InvalidBet()
                        elif bet == 0 :
                            stat = 'f'
                            break
                        else :
                            currentPot += bet
                            player.bankroll -= bet
                            break
                    except InvalidBet:
                        print ('Please enter a valid number greater than or equal to 0')
                    except ValueError:
                        print ('Please enter a valid number greater than or equal to 0')

                #Create Dealer's Bet
                if stat == 'c' :
                    dealerBetAmount = dealerBet(dealer, player, bet)
                    currentPot += dealerBetAmount
                    dealer.bankroll -= dealerBetAmount
                    printBoard(player, dealer, currentPot, winAmount)

                #Player turn to hit or stay
                while (stat == 'c') :
                    try :
                        decision = raw_input("What would you like to do Hit or Stand or Dealer Hand: ").lower()
                        if decision != 'hit' and decision != 'stand' and decision != 'dealer hand':
                            raise InvalidInput()
                    except InvalidInput:
                        print ("Please enter 'Hit' or 'Stand' or 'Dealer Hand' without the quotations")
                        continue
                    if decision == 'hit':
                        player.hand.append(deck.hit())
                        printBoard(player, dealer, currentPot, winAmount)
                        total = sumHand(player.hand)
                        if total > 21 :
                            stat = 'b'
                        elif total < 21 :
                            stat = 'c'
                        else :
                            stat = 'w'
                    elif decision == 'stand' :
                        stat = 's'
                    else :
                        dealerPrintHand = [str(card) for card in dealer.hand]
                        dealerPrintHand[0] = '*' 
                        print (dealerPrintHand)
                        continue
                printBoard(player, dealer, currentPot, winAmount)

                #Create Dealer's Play
                if stat != 'f' and stat != 'b':
                    stat = dealerPlay(dealer, player, currentPot, winAmount, deck, stat)

                #Print results
                playerPrintHand = [str(card) for card in player.hand]
                dealerPrintHand = [str(card) for card in dealer.hand]
                print("Player hand Value : %d") %(sumHand(player.hand))
                print(playerPrintHand)
                print("\nDealer Hand Value : %d") %(sumHand(dealer.hand))
                print(dealerPrintHand)
                
                if stat == 'f' :
                    print ("{x} folded, House wins").format(x = player.pName)
                    player.losses += 1
                    dealer.wins += 1
                elif stat == 'D' or stat == 'b' :
                    print ("House wins {x}").format(x = currentPot)
                    player.losses += 1
                    dealer.wins += 1
                    dealer.bankroll += currentPot
                else :
                    print ("{x} wins {p}").format(x = player.pName, p = currentPot)
                    player.wins += 1
                    player.bankroll += currentPot
                    dealer.losses += 1
                
                #Clear Hands
                player.clear()
                dealer.clear()

            if player.bankroll <= 0 :
                print ("You lose the game\nTotal Losses = {x}\nTotal Wins = {y}").format(x = player.losses, y = player.wins)
            else :
                print ("You won the game\nTotal Wins = {x}\nTotal Losses = {y}").format(x = player.wins, y = player.losses)
            
            playAgain = ""
            while True :
                try :
                    playAgain = raw_input("Would you like to play again yes or no :").lower()
                    if playAgain != 'yes' and playAgain != 'no' :
                        raise InvalidInput()
                    else :
                        break
                except InvalidInput :
                    print("Please enter yes or no")
                    continue
            if playAgain == 'yes' :
                raise PlayAgain()
            else :
                break
        except PlayAgain:
            print ("Here we go")
            continue
    print("Thank you for playing come again!")

In [11]:
blackjack()

Please enter your name nick
How much money would you like to start with? 100
Thank you nick your starting bank will be 100
What dollar amount would you like to play to 300
Thank you the game will be played until you get 300 or more or go broke
Dealer wins ties
			|
			|
			|
			|
			|
-------------------------------------------------------------------------
Current Pot : 0  |        | Dealer |          Dealer Wins   = 0
-------------------~                             Dealer Losses = 0
                                                 Dealer Bank   = 900
                          Amount to Win                             
                                300                                 
nick Wins   = 0                                                    
nick Losses = 0                                                    
nick Bank   = 100                                                    
Your Hand :                                                         
['6 of Spade', '3 of Club']